In [78]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

In [3]:
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.metrics import silhouette_score

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances, euclidean_distances


%matplotlib inline
plt.style.use('fivethirtyeight')

In [4]:
data = pd.read_csv("data.csv", index_col=0)

data.columns = data.columns.str.lower().str.replace(" ", "_")

bad_cols = ["club_logo", "photo", "flag", "loaned_from"]

data = data.drop(bad_cols, axis = 1)
data = data.query("position != 'GK'")
gk_cols = [i for i in data.columns if "gk" in i]

data = data.drop(gk_cols, axis = 1)
data = data.set_index("id")

In [5]:
top5 = np.load("top_5_clubs.pkl")

In [7]:
len(top5)

102

In [9]:
data = data[data.club.isin(top5)].copy()

In [14]:
# data = data[data.overall >= 70].copy()

In [10]:
attribute_cols = data.columns[48:-1].tolist()
features = data[attribute_cols].copy()
features.dropna(inplace=True)


In [15]:
features["tackling"] = (features.standingtackle + features.slidingtackle)/2.
features.drop(["standingtackle", "slidingtackle"], axis = 1, inplace=True)

In [16]:
features.columns

Index(['crossing', 'finishing', 'headingaccuracy', 'volleys', 'dribbling',
       'curve', 'fkaccuracy', 'ballcontrol', 'acceleration', 'sprintspeed',
       'agility', 'reactions', 'balance', 'shotpower', 'jumping', 'stamina',
       'strength', 'longshots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'composure', 'marking', 'passing', 'tackling'],
      dtype='object')

In [17]:
cols = ["finishing", "passing", "dribbling", "curve", "ballcontrol", "tackling", "balance", "marking", "agility"]

In [18]:
X = features[cols].copy()

In [22]:
name_id_dict = data.name.to_dict()

In [30]:
hazard_id = 183277
salah_id = 209331

In [31]:
# name_id_dict

In [40]:
trace_data_1 = X.loc[hazard_id].tolist()
trace_data_2 = X.loc[salah_id].tolist()

trace1 = go.Scatterpolar(r = trace_data_1 + [trace_data_1[0]], theta = cols + [cols[0]], fill = "toself",
                        name = name_id_dict[hazard_id], opacity = .5)

trace2 = go.Scatterpolar(r = trace_data_2 + [trace_data_2[0]], theta = cols + [cols[0]], fill = "toself",
                        name = name_id_dict[salah_id], opacity = .5)

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 100]
    )
  ),
  showlegend = True
)

In [69]:
# fig = go.Figure(data = [trace1, trace2], layout= layout)
# py.iplot(fig)

In [70]:
# fig.to_dict()

In [43]:
import dash_core_components as dcc

In [45]:
# help(dcc.Dropdown)

In [47]:
{"label":i for i in data.name}

{'label': 'A. Eleftheriou'}

In [63]:
options = []

In [64]:
for i in data.name.unique():
    d = {"label":i, "value":i}
    options.append(d)

In [56]:
names = data.name

In [61]:
data.loc[a.duplicated(), "name"] = a[a.duplicated()] + ", " + data.loc[a.duplicated(), "club"]

In [62]:
data.name.nunique()

2705

In [68]:
data[data.name == "Danilo"].index[0]

199304

In [71]:
hist_color = "#D91E36"

In [162]:
salah_data = X.loc[salah_id]

In [237]:
grid = []

for i in n:
    for e in n:
        grid.append([i, e])
        
grid

[[1, 1], [1, 2], [1, 3], [2, 1], [2, 2], [2, 3], [3, 1], [3, 2], [3, 3]]

In [236]:
n

[1, 2, 3]

In [226]:
# fig = tools.make_subplots(rows=3, cols=3, subplot_titles=cols);
# for i in range(len(cols)):
#     c = cols[i]
#     trace = go.Histogram(x = X[c], name = c, marker = dict(color = hist_color), 
# #                          nbinsx = X[c].nunique() + 1,
#                          xbins = dict(start = 0, end = 100, size = 1)
#                         )
#     fig.append_trace(trace,grid[i][0], grid[i][1])
#     value = salah_data.loc[c]
#     sd = go.Scatter(x =[value, value], y = [0, 100] , 
#                     marker = dict(color = "black"), mode = "lines")
#     fig.append_trace(sd,grid[i][0], grid[i][1])
    

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]
[ (3,1) x7,y7 ]  [ (3,2) x8,y8 ]  [ (3,3) x9,y9 ]



In [229]:
fig = tools.make_subplots(rows=3, cols=3, subplot_titles=cols);
for i in range(len(cols)):
    c = cols[i]
    
    vc = X[c].value_counts().reindex(np.arange(X[c].min()- 1, X[c].max() + 1))
    
    trace = go.Bar(x = vc.index, y = vc.values, name = c, marker = dict(color = hist_color))
    
    
    fig.append_trace(trace,grid[i][0], grid[i][1])
    value = salah_data.loc[c]
    sd = go.Scatter(x =[value, value], y = [0, vc.max()] , 
                    marker = dict(color = "black"), mode = "lines")
    fig.append_trace(sd,grid[i][0], grid[i][1])
    

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]
[ (3,1) x7,y7 ]  [ (3,2) x8,y8 ]  [ (3,3) x9,y9 ]



In [230]:
fig["layout"].update(showlegend = False, height=750, width=750,
                     title='FIFA Histograms');

In [232]:
# py.iplot(fig)

In [ ]:
dcc.Dropdown()